In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb

# 데이터 불러오기

In [4]:
energy = pd.read_csv('energy.csv')
dangjin_fcst = pd.read_csv('new_dj_fcst.csv')


In [5]:
energy.head(10)

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
0,2018-03-01 1:00,0.0,0.0,0,0
1,2018-03-01 2:00,0.0,0.0,0,0
2,2018-03-01 3:00,0.0,0.0,0,0
3,2018-03-01 4:00,0.0,0.0,0,0
4,2018-03-01 5:00,0.0,0.0,0,0
5,2018-03-01 6:00,0.0,0.0,0,0
6,2018-03-01 7:00,0.0,0.0,0,0
7,2018-03-01 8:00,0.0,0.0,0,4
8,2018-03-01 9:00,36.0,33.0,37,35
9,2018-03-01 10:00,313.0,209.0,318,71


In [16]:
dangjin_fcst.loc[25608,:]

Forecast_time    2021-02-01 00:00:00
Temperature                        6
Humidity                          80
WindSpeed                        5.6
WindDirection                    177
Cloud                              4
Name: 25608, dtype: object

In [26]:
dangjin_fcst=pd.DataFrame(dangjin_fcst)
dangjin_fcst['Forecast_time'][25608]

'2021-02-01 00:00:00'

# 학습 데이터 전처리

In [27]:
def train_datast(energy_df,fcst_df,target):
    # 일기 예보 있는 날짜만 선택 2018-03-02 이후
    energy=energy_df.loc[24:]
    energy.index=range(energy.shape[0])
    
    # 발전량 데이터가 있는 날짜만 선택 2021-02-01이전까지
    fcst=fcst_df.loc[:25608-1]
    fcst.index=range(fcst.shape[0])
    
    #발전량과 일기예보 연결
    concat_df=pd.concat([energy,fcst],axis=1)
    
    # 예보 시간 및 날짜 정보 feature로 추가
    concat_df['date']=concat_df['Forecast_time'].str.split('').str[0]
    concat_df['hour']=concat_df['Forecast_time'].str.split('').str[1].str.split(':').str[0].astype(int)
    
    concat_df['year']=concat_df['date'].str.split('-').str[0].astype(int)
    concat_df['month']=concat_df['date'].str.split('-').str[1].astype(int)
    concat_df['day']=concat_df['date'].str.split('-').str[2].astype(int)
    
    
    # 예보 시간, 날짜, 기상 예보 및 발전량 선택
    feature_df = concat_df[['year', 'month', 'day', 'hour', 'Temperature', 'Humidity', 'WindSpeed', 'WindDirection', 'Cloud', target]]
    
    # 마지막 30일을 검증데이터셋으로 나머지를 학습 데이터 셋으로 선택
    train_df=feature_df.loc[:-24*30]
    val_df=feature_df.loc[-24*30:]
    
    # 발전량이 0인 데이터를 제외
    train_df=train_df[train_df[target]!=0]
    
    train_x=train_df.loc[:,'year':'cloud'].to_numpy()
    train_y=train_df[target].to_numpy()
    
    
    val_x = val_df.loc[:, 'year':'Cloud'].to_numpy()
    val_y = val_df[target].to_numpy()
    
    return train_x,train_y,val_x,val_y
    
    
    

## LGBM Custom Metric


In [28]:
def nmae_10(y_pred,dataset):
    y_true=dataset.get_label()
     
    absolute_error=abs(y_true-y_pred)
    absolute_error/=capacity
    
    target_idx=np.where(y_ture>=capacity*0.1)
    nmae=100*absolute_error[target_idx].mean()
    
    return 'score', nmae, False

## Validation Metric


In [29]:
def sola_nmae(answer, pred):
    absolute_error = np.abs(answer - pred)
    
    absolute_error /= capacity
    
    target_idx = np.where(answer>=capacity*0.1)
    
    nmae = 100 * absolute_error[target_idx].mean()
    
    return nmae

## 당진수상태양광 예측 모델 학습